## **Travelling Salesperson Problem**

In [634]:
import pandas as pd
from icecream import ic
import numpy as np
import random
import math
from geopy.distance import geodesic

## Approach 1: **Best greedy solution**
This first trivial approach is based on a greedy algorithm where the local optimum, at each step, is represented by the closest city among the ones that have not been visited yet. The greedy process is repeated as many times as the number of cities, and each time the path starts from a different city.<br>
The one greedy solution that is chosen as the final one is the one whose overall distance is shorter.<br>

In [635]:
class City:
    def __init__(self, name, x, y):
        self.name = name
        self.x = x
        self.y = y

source = pd.read_csv("cities/italy.csv", names=["city", "x", "y"])
cities = list()

# initialization of the cities list with City objects
for index, row in source.iterrows():
    cities.append(City(row["city"], row["x"], row["y"]))

NUM_CITIES = len(cities)

The distance between the cities can be measured either using euclidean distance or using `geodesic()` function (based on latitude and longitude). The code is parametrized so that the distance measurement method can be changed by setting the `DISTANCE` constant to `"euclidean"` or `"geodesic"`.<br>
**NOTE**: changing the type of distance does not impact on the algorithm strategy or on final solution.

In [636]:
DISTANCE = "geodesic"
# the function calculates the euclidean distance between two cities
def euclidean_distance(city_1, city_2):
    return math.sqrt(math.pow((city_1.x - city_2.x), 2) + math.pow((city_1.y - city_2.y), 2))

# matrix containing the precalculated geodesic distances
DIST_MATRIX = np.zeros((len(cities), len(cities)))
# the function calculates the geodesic distance between two cities
def geodesic_distance(city_1, city_2):
    return DIST_MATRIX[cities.index(city_1), cities.index(city_2)]

# initialization of the geodesic distances matrix
if DISTANCE == "geodesic":
    for i in range(len(cities)):
        for j in range(len(cities)):
            point_1 = (cities[i].x, cities[i].y)
            point_2 = (cities[j].x, cities[j].y)
            DIST_MATRIX[i, j] = geodesic(point_1, point_2).kilometers

# the function returns the distance among two cities according to the selected distance method
def get_distance(city_1, city_2):
    if DISTANCE == "euclidean":
        return euclidean_distance(city_1, city_2)
    else:
        return geodesic_distance(city_1, city_2)

In [637]:
# the function checks whether a candidate city is already in the solution (i.e. it has already been visited)
def not_in_solution(city, solution):
    flag = True
    for sol_city in (solution[0]):
        if sol_city.name == city.name:
            flag = False
            break

    return flag

# the function returns the available city that is closer to the last one visited
def get_closest_city(solution):
    closer_dist = float("inf")
    closer_city = None
    for city in cities:
        dist = get_distance(solution[0][-1], city)
        if dist != 0 and dist < closer_dist and not_in_solution(city, solution):
            closer_city = city
            closer_dist = dist
    
    solution[0].append(closer_city)
    new_solution = (solution[0], solution[1] + closer_dist)
    return new_solution

# the function prints the final result
def print_solution(solution):
    print("PATH:")
    for i in range(0, NUM_CITIES - 1):
        print("(" + solution[i].name + " -> " + solution[i + 1].name + ") DSITANCE: " + str(get_distance(solution[i], solution[i + 1])))
    print("(" + solution[-1].name + " -> " + solution[0].name + ") DSITANCE: " + str(get_distance(solution[-1], solution[0])))
    print("TOTAL DISTANCE: " + str(recalculate_distance(solution)))

# the function recalculates the overall path distance after the city swap performed in the 2-opt
def recalculate_distance(path):
    dist = 0.0
    for i in range(0, NUM_CITIES - 1):
        dist += get_distance(path[i], path[i + 1])
    dist += get_distance(path[-1], path[0])
    return dist

In [638]:
best_result = float("inf")

# the greedy algorithm is repeated each time with a different starting city
for i in range(0, NUM_CITIES):
    starting_city = cities[i]
    # initialization of the solution
    candidate_solution = ([starting_city], 0.0)

    solution = candidate_solution
    # at each iteration a new city is added to the path; the city is chosen so that it is the closest
    # one to the last visited city
    for _ in range(NUM_CITIES - 1):
        solution = get_closest_city(solution)

    # the distance needed to come back to the starting city is added to the total path distance
    solution = (solution[0], solution[1] + get_distance(starting_city, solution[0][-1]))
    # the 2-opt technique is performed on the greedy solution in order to try to improve it
    # solution = two_opt(solution)
    # if the current greedy solution is the best one then it's set as the final one
    if solution[1] < best_result:
        best_result = solution[1]
        best_greedy = solution

print_solution(best_greedy[0])

PATH:
(Ancona -> Rimini) DSITANCE: 90.5967461797061
(Rimini -> Forlì) DSITANCE: 46.723760122246745
(Forlì -> Ravenna) DSITANCE: 26.45904737732812
(Ravenna -> Ferrara) DSITANCE: 66.67171194357726
(Ferrara -> Bologna) DSITANCE: 43.42728615199659
(Bologna -> Modena) DSITANCE: 37.29202792818561
(Modena -> Reggio nell'Emilia) DSITANCE: 23.939721051289947
(Reggio nell'Emilia -> Parma) DSITANCE: 26.942823860477837
(Parma -> Piacenza) DSITANCE: 57.65426248112829
(Piacenza -> Milan) DSITANCE: 60.64923118840254
(Milan -> Monza) DSITANCE: 14.50856854090449
(Monza -> Bergamo) DSITANCE: 33.91792128864521
(Bergamo -> Brescia) DSITANCE: 46.01771032749969
(Brescia -> Verona) DSITANCE: 61.41588754627765
(Verona -> Vicenza) DSITANCE: 44.69517255475827
(Vicenza -> Padua) DSITANCE: 30.129893989361843
(Padua -> Venice) DSITANCE: 36.07208970412391
(Venice -> Trieste) DSITANCE: 115.09339706818346
(Trieste -> Bolzano) DSITANCE: 209.6848929934776
(Bolzano -> Trento) DSITANCE: 49.93679674999438
(Trento -> Novar

## Approach 2: **Best greedy solution with 2-opt**
In this second approach we try to improve the best greedy solution by adding an iterative 2-opt mutation to the solution for a fixed number of times.<br>
The implemented fitness functions are two:<br>
- `fitness()`: based on the total distance of the path
- `fitness_var()`: based on the variance of the length of the segments connecting the cities along the path. This is made in order to make the overall path more "balanced" with respect to the distances and also in order to try to fix the greedy scenario where the local optimality (closest city) does not necessarily correspond to the global optimality (shortest path).

In [639]:
# fitness function baed on distance
def fitness(solution):
    return -recalculate_distance(solution)

# fitness function based on variance, calculated as (x - u)^2 for each city x, where u is the 
# value of the average length of the segments connecting the cities in the current solution path
def fitness_var(solution):
    sum = 0.0
    for i in range(0, NUM_CITIES - 1):
        sum += get_distance(solution[i], solution[i + 1])
    sum += get_distance(solution[-1], solution[0])
    avg = sum / NUM_CITIES

    var = 0.0
    for i in range(0, NUM_CITIES - 1):
        var += math.pow((get_distance(solution[i], solution[i + 1]) - avg), 2)
    var += math.pow((get_distance(solution[-1], solution[0]) - avg), 2)

    return (-sum, - var)


# the 2-opt function performs the swap of two random cities
def two_opt(solution):
    # selection of the two cities to be swapped
    edge_1 = random.randint(0, (NUM_CITIES - 1))
    edge_2 = random.randint(0, (NUM_CITIES - 1))
    while edge_2 == edge_1:
        edge_2 = random.randint(0, (NUM_CITIES - 1))
    
    # the cities are swapped and the new distance is calculated
    new_path = solution.copy()
    temp = new_path[edge_1]
    new_path[edge_1] = new_path[edge_2]
    new_path[edge_2] = temp
    new_solution = new_path
            
    return new_solution


# iterative process
# the starting solution is the best greedy solution
solution = best_greedy[0].copy()
for _ in range(1000):
    new_solution = two_opt(solution)

    if fitness_var(new_solution) > fitness_var(solution):
        solution = new_solution

print_solution(solution)

PATH:
(Ancona -> Rimini) DSITANCE: 90.5967461797061
(Rimini -> Forlì) DSITANCE: 46.723760122246745
(Forlì -> Ravenna) DSITANCE: 26.45904737732812
(Ravenna -> Ferrara) DSITANCE: 66.67171194357726
(Ferrara -> Bologna) DSITANCE: 43.42728615199659
(Bologna -> Modena) DSITANCE: 37.29202792818561
(Modena -> Reggio nell'Emilia) DSITANCE: 23.939721051289947
(Reggio nell'Emilia -> Parma) DSITANCE: 26.942823860477837
(Parma -> Piacenza) DSITANCE: 57.65426248112829
(Piacenza -> Milan) DSITANCE: 60.64923118840254
(Milan -> Monza) DSITANCE: 14.50856854090449
(Monza -> Bergamo) DSITANCE: 33.91792128864521
(Bergamo -> Brescia) DSITANCE: 46.01771032749969
(Brescia -> Verona) DSITANCE: 61.41588754627765
(Verona -> Vicenza) DSITANCE: 44.69517255475827
(Vicenza -> Padua) DSITANCE: 30.129893989361843
(Padua -> Venice) DSITANCE: 36.07208970412391
(Venice -> Trieste) DSITANCE: 115.09339706818346
(Trieste -> Bolzano) DSITANCE: 209.6848929934776
(Bolzano -> Trento) DSITANCE: 49.93679674999438
(Trento -> Novar

## Approach 3: **Hill climbing with 2-opt and simulated annealing**
This third approach starts from a random solution and tries to iprove it with a 2-opt tweak. The algorithm also implements simulated annealing using a "temeperature" level that represents the number of times the tweak is applied to the current solution (i.e. if the temperature is low the solution is slightly changed, while if the temperature is high the solution is highly changed as well).

In [640]:
temperature = 1
BUFFER_SIZE = 4
buffer = list()

# the function performs as many random city swaps as the value of the temperature
def two_opt_sa(solution):
    for _ in range(int(temperature)):
        # selection of the two cities to be swapped
        edge_1 = random.randint(0, (NUM_CITIES - 1))
        edge_2 = random.randint(0, (NUM_CITIES - 1))
        while edge_2 == edge_1:
            edge_2 = random.randint(0, (NUM_CITIES - 1))
        
        # the cities are swapped and the new distance is calculated
        new_path = solution.copy()
        temp = new_path[edge_1]
        new_path[edge_1] = new_path[edge_2]
        new_path[edge_2] = temp
        new_solution = new_path

        solution = new_solution
            
    return solution


# we start from a random solution
random_path = best_greedy[0].copy()
random.shuffle(random_path)
solution = random_path
print_solution(solution)

# iterative process
for _ in range(1000):
    new_solution = two_opt_sa(solution)

    buffer.append(fitness(new_solution) > fitness(solution))
    buffer = buffer[-BUFFER_SIZE:]

    # variation of the temperature
    if sum(buffer) > BUFFER_SIZE / 2 and temperature > 1:
        temperature -= 1
    elif sum(buffer) < BUFFER_SIZE / 2:
        temperature += 1

    if fitness(new_solution) > fitness(solution):
        solution = new_solution
    
print_solution(solution)

PATH:
(Bolzano -> Venice) DSITANCE: 141.11716133038544
(Venice -> Ravenna) DSITANCE: 112.64069747773023
(Ravenna -> Leghorn) DSITANCE: 181.16841963801093
(Leghorn -> Giugliano in Campania) DSITANCE: 433.2484955085369
(Giugliano in Campania -> Perugia) DSITANCE: 284.33831841710895
(Perugia -> Bari) DSITANCE: 431.3370389983044
(Bari -> Vicenza) DSITANCE: 654.7220646876538
(Vicenza -> Reggio nell'Emilia) DSITANCE: 117.64010366532213
(Reggio nell'Emilia -> Rimini) DSITANCE: 170.62507125856942
(Rimini -> Trieste) DSITANCE: 200.5394101874627
(Trieste -> Reggio di Calabria) DSITANCE: 851.8202258824483
(Reggio di Calabria -> Monza) DSITANCE: 983.7020377360125
(Monza -> Syracuse) DSITANCE: 1070.3353744883243
(Syracuse -> Forlì) DSITANCE: 840.3137488802347
(Forlì -> Pescara) DSITANCE: 263.5770276196447
(Pescara -> Turin) DSITANCE: 600.7042049362549
(Turin -> Latina) DSITANCE: 582.6203924306731
(Latina -> Catania) DSITANCE: 479.3019034969043
(Catania -> Prato) DSITANCE: 785.4054494528867
(Prato -

## Approach 4: **Best greedy with insert mutation**
This fourth approach, like the second one, tries to improve the best greedy solution by iteratively applying a mutation, which, in this case, is the insert mutation.<br>
The fitness function is again based on the total distance of the path.

In [641]:
# mutation function
def insert_mutation(solution):
    edge_1 = random.randint(0, (NUM_CITIES - 2))
    edge_2 = random.randint(edge_1 + 1, (NUM_CITIES - 1))

    new_path = list()
    new_path.extend(solution[0: edge_1 + 1])
    new_path.append(solution[edge_2])
    new_path.extend(solution[edge_1 + 1: edge_2])
    new_path.extend(solution[edge_2 + 1: NUM_CITIES])
    new_solution = new_path

    return new_solution


# we start from the best greedy solution
solution = best_greedy[0].copy()
# iterative process
for _ in range(1000):
    new_solution = insert_mutation(solution)

    if fitness(new_solution) > fitness(solution):
        solution = new_solution

print_solution(solution)

PATH:
(Ancona -> Rimini) DSITANCE: 90.5967461797061
(Rimini -> Forlì) DSITANCE: 46.723760122246745
(Forlì -> Ravenna) DSITANCE: 26.45904737732812
(Ravenna -> Ferrara) DSITANCE: 66.67171194357726
(Ferrara -> Bologna) DSITANCE: 43.42728615199659
(Bologna -> Modena) DSITANCE: 37.29202792818561
(Modena -> Reggio nell'Emilia) DSITANCE: 23.939721051289947
(Reggio nell'Emilia -> Parma) DSITANCE: 26.942823860477837
(Parma -> Piacenza) DSITANCE: 57.65426248112829
(Piacenza -> Milan) DSITANCE: 60.64923118840254
(Milan -> Monza) DSITANCE: 14.50856854090449
(Monza -> Bergamo) DSITANCE: 33.91792128864521
(Bergamo -> Brescia) DSITANCE: 46.01771032749969
(Brescia -> Verona) DSITANCE: 61.41588754627765
(Verona -> Vicenza) DSITANCE: 44.69517255475827
(Vicenza -> Padua) DSITANCE: 30.129893989361843
(Padua -> Venice) DSITANCE: 36.07208970412391
(Venice -> Trieste) DSITANCE: 115.09339706818346
(Trieste -> Bolzano) DSITANCE: 209.6848929934776
(Bolzano -> Trento) DSITANCE: 49.93679674999438
(Trento -> Novar

## Approach 5: **Hill climbing with insert mutation and simulated annealing**
This fifth approach, like the third one, starts from a random solution and improves it using simulated annealing strategy with insert mutation as a tweak function.<br>
The fitness function is again based on the total distance of the path.

In [642]:
temperature = 1
BUFFER_SIZE = 4
buffer = list()


# mutation function
def insert_mutation_sa(solution):
    for _ in range(temperature):
        edge_1 = random.randint(0, (NUM_CITIES - 2))
        edge_2 = random.randint(edge_1 + 1, (NUM_CITIES - 1))

        new_path = list()
        new_path.extend(solution[0: edge_1 + 1])
        new_path.append(solution[edge_2])
        new_path.extend(solution[edge_1 + 1: edge_2])
        new_path.extend(solution[edge_2 + 1: NUM_CITIES])

    return new_path


# we start from a random solution
random_path = best_greedy[0].copy()
random.shuffle(random_path)
solution = random_path
print_solution(solution)

# iterative process
for _ in range(1000):
    new_solution = insert_mutation_sa(solution)

    buffer.append(fitness(new_solution) > fitness(solution))
    buffer = buffer[-BUFFER_SIZE:]

    if sum(buffer) > BUFFER_SIZE / 4 and temperature > 1:
        temperature -= 1
    elif sum(buffer) < BUFFER_SIZE / 4:
        temperature += 1

    if fitness(new_solution) > fitness(solution):
        solution = new_solution
    
print_solution(solution)

PATH:
(Perugia -> Ferrara) DSITANCE: 202.15290561946034
(Ferrara -> Monza) DSITANCE: 201.3737313208582
(Monza -> Ravenna) DSITANCE: 265.2187980495849
(Ravenna -> Novara) DSITANCE: 305.58891262615623
(Novara -> Catania) DSITANCE: 1033.95619199117
(Catania -> Bari) DSITANCE: 430.50090746323104
(Bari -> Rimini) DSITANCE: 480.73854159440583
(Rimini -> Reggio di Calabria) DSITANCE: 709.5112921234869
(Reggio di Calabria -> Sassari) DSITANCE: 675.9990106220904
(Sassari -> Ancona) DSITANCE: 517.7896360136997
(Ancona -> Bergamo) DSITANCE: 383.0204391314121
(Bergamo -> Taranto) DSITANCE: 845.5713728385285
(Taranto -> Venice) DSITANCE: 680.085743308479
(Venice -> Messina) DSITANCE: 847.2962295195107
(Messina -> Modena) DSITANCE: 814.863584686536
(Modena -> Genoa) DSITANCE: 160.21649089511217
(Genoa -> Andria) DSITANCE: 698.018461099247
(Andria -> Naples) DSITANCE: 175.00968691893462
(Naples -> Latina) DSITANCE: 134.743654502029
(Latina -> Turin) DSITANCE: 582.6203924306731
(Turin -> Rome) DSITANC

The following one is a fitness function based on the minimization of the intersections between the segments connecting the different cities in order to "simplify" the path as much as possible. This one was not implemented as it showed to be highly inefficient.

In [643]:
def orientation(p, q, r):
    val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
    if val == 0:
        return 0  
    elif val > 0:
        return 1  
    else:
        return 2 

def on_segment(p, q, r):
    return (min(p[0], r[0]) <= q[0] <= max(p[0], r[0]) and
            min(p[1], r[1]) <= q[1] <= max(p[1], r[1]))

def do_intersect(A, B, C, D):
    o1 = orientation(A, B, C)
    o2 = orientation(A, B, D)
    o3 = orientation(C, D, A)
    o4 = orientation(C, D, B)

    if o1 != o2 and o3 != o4:
        return True

    if o1 == 0 and on_segment(A, C, B):
        return True
    if o2 == 0 and on_segment(A, D, B):
        return True
    if o3 == 0 and on_segment(C, A, D):
        return True
    if o4 == 0 and on_segment(C, B, D):
        return True

    return False

def fitness_intersection(solution):
    num = 0
    for i in range(0, NUM_CITIES - 1):
        for j in range(0, NUM_CITIES - 1):
            if i != j:
                A = (solution[0][i].x, solution[0][i].y)
                B = (solution[0][i + 1].x, solution[0][i + 1].y)
                C = (solution[0][j].x, solution[0][j].y)
                D = (solution[0][j + 1].x, solution[0][j + 1].y)
                if do_intersect(A, B, C, D):
                    num += 1
    return -num